# HHVM

## 背景介绍

HHVM 是 Facebook (现 Meta) 开发的高性能 PHP 虚拟机，宣称达到了官方解释器的 9x 性能

### 为什么会有 HHVM

#### 脚本语言


##### Pros

一般我们使用脚本语言 （Perl，Python，PHP，JavaScript）是为了以下几个目的

1. 大部分的脚本语言都拥有较为完备的外部库，能够帮助开发者快速的开发/测试

   - 使用 Python 作为 ebt 的技术栈也是因为 `numpy`, `pandas` 等数据科学库的支持比别的编程语言更加的完备

2. 动态语言的特性使得开发过程变得异常轻松，可以最大程度的实现可复用性和多态性，打个比方

   ```python
   def evaluate(model_impl, params):
       return model_impl.calculate(params)


   class Model(object):
       def calculate(params):
           sum_val = 0
           for param in params:
               sum_val += param

           return sum_val
   ```

   ```cpp
   class IModel {
       public:
           virtual double calculate(const vector<double> &params) = 0;
           virtual int calculate(const vector<int> &params) = 0;
   }

   class Model : public IModel {
       public:
           double calculate(const vector<double> &params) {
               double sum_val = 0;
               for (vector<double>::iterator it = params.begin(); it != params.end(); ++it) {
                   sum_val += *it;
               }
               return sum_val;
           }

           int calculate(const vector<int> &params) {
               int sum_val = 0;
               for (vector<int>::iterator it = params.begin(); it != params.end(); ++it) {
                   sum_val += *it;
               }
               return sum_val;
           }
   }

   double evaluate(IModel* model_impl, const vector<double> &params) {
       return model_impl->calculate(params);
   }

   int evaluate(IModel* model_impl, const vector<int> &params) {
       return model_impl->calculate(params);
   }
   ```

   - 模版

   ```cpp
   // This is ok but template is not a general feature for all static typed language
   template <typename T>
   T evaluate(IModel* model_impl, const vector<T> &params) {
       return model_impl->calculate<T>(params);
   }
   ```

3. 动态语言完全是解释执行，调试成本较低。每当改动源码有所改动后，程序重新运行更加直接，只需要解释器重新读取源码即可。编译性语言需要更多的步骤与时间，例如 C++，为了从源码生成可执行程序需要 链接静态库 -> .obj -> 链接动态库 -> 可执行程序。如果是大型项目开发的话这一步骤甚至会花费几个小时。而解释执行的程序可以不需要这些步骤直接重新运行


##### Cons

但是对于有较高性能需求的 situation，编译执行反而会成为拖累。

> Although easy to implement, interpreters are generally slow, which makes scripting language prohibitive for implementing large, CPU-intensive applications. (Zhao, 2021)

Debian 有一个 [benchmark game](https://benchmarksgame-team.pages.debian.net/benchmarksgame/index.html)，比较了目前比较常见的几种编程语言的运行速度/内存占用/源码大小

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
benchmarks = pd.read_csv('./data/programming_language_benchmarks_game_all_measurements.csv')

benchmarks.head(10)

name  lang  id   n  size(B)  cpu(s)  mem(KB)  status  \
0  binarytrees  rust   2   7      721   0.004        0       0   
1  binarytrees  rust   2  14      721   0.027        0       0   
2  binarytrees  rust   2  21      721   4.286   199808       0   
3  binarytrees  rust   2  21      721   4.387   175268       0   
4  binarytrees  rust   2  21      721   4.414   199704       0   
5  binarytrees  rust   2  21      721   4.271   199760       0   
6  binarytrees  rust   2  21      721   4.315   199756       0   
7  binarytrees  rust   2  21      721   4.297   199848       0   
8  binarytrees   ghc   8   7      799   0.009        8       0   
9  binarytrees   ghc   8  14      799   0.047        0       0   

               load  elapsed(s)  busy(s)  
0     0% 33% 0% 75%       0.091    0.098  
1                 %       0.000    0.000  
2   85% 88% 99% 87%       1.206    4.329  
3   87% 86% 98% 88%       1.235    4.437  
4   87% 99% 88% 87%       1.234    4.457  
5   88% 87% 99% 87%       1.201    4.333  
6   87% 86% 99% 85%       1.220    4.365  
7   88% 88% 98% 88%       1.207    4.369  
8   67% 10% 60% 23%       0.207    0.330  
9  50% 100% 50% 50%       0.024    0.061

In [2]:
compile_lang_lst = ['clang', 'csharpcore', 'csharppgo', 'gcc', 'gfortran', 'go', 'gpp', 'java', 'rust', 'swift']
interpreter_lang_lst = ['node', 'perl', 'php', 'python3']

def boxplot_by_lang(data: pd.DataFrame, colname: str) -> None:
    fig, ax = plt.subplots()
    index = 1
    for lang in compile_lang_lst:
        ax.boxplot(data[data['lang'] == lang][colname],
                   positions=[index],
                   labels=[lang],
                   boxprops=dict(color='blue'))
        index += 1

    for lang in interpreter_lang_lst:
        ax.boxplot(data[data['lang'] == lang][colname],
                   positions=[index],
                   labels=[lang],
                   boxprops=dict(color='green'))
        index += 1

    ax.set_title(colname, fontsize=15)
    ax.tick_params(axis='x', labelrotation=45)
    fig.set_size_inches(10, 6)

In [3]:
filtered = benchmarks[(benchmarks['status'] == 0) & (benchmarks['name'] == 'binarytrees') & (benchmarks['n'] == 21)].reset_index()
boxplot_by_lang(data=filtered, colname='elapsed(s)')

In [4]:
boxplot_by_lang(data=filtered, colname='mem(KB)')


In [5]:
boxplot_by_lang(data=filtered, colname='size(B)')


通过以上数据可以显然看出与编译执行语言相比，解释执行的语言在 CPU 的处理性能有明显的优势，部分编译执行的语言在内存处理（申请与回收）上也有着异常优秀的表现。

像 Meta 这样的巨型公司需要 host 的服务器是也是巨型的


<img src='./images/faceboook_datacenter_oregon.png' alt='facebook_datacenter' width='1000' />

上图是 Meta 在 Oregon 的数据中心，据说这两个 building 的造价就高达了 *$750M* (约合 *¥47.78 亿*)，并且在 2020 年中 Meta 又在边上造了两个

如此巨型的数据中心的一大作用就是用来做 Facebook 的服务器主机，所以为了优化服务器的物理成本，从代码上优化服务器性能是必然的。

我们都知道 Facebook 是用 PHP 实现的，据 HHVM 的项目主持者之一的 Keith Adams 所说，Facebook 有约莫 $2 \times 10^7$ 行 PHP 代码 （2012年）。Facebook 的开发者在检查之后发现自己服务器的性能问题很大一部分就是资源的消耗就在 PHP 的解释器本身上，所以需要考虑的就是如何优化 PHP 的性能

### 如何优化 PHP 的性能

1. 使用性能更好的语言重写服务端，如 C++，GO，Java
    - 重构2千万行代码？算了算了

2. 使用 RPC 将部分业务功能独立，减少 PHP 的处理，比如 Twitter 就将自己的很多业务逻辑从 Ruby on Rails 转为了 Java 和 Scala (前端由 node + react 独立实现)

<img src='./images/twitter_tech_stack.webp' alt='twitter_tech_stack' width="1000" />

   - RPC 框架  
   <img src='./images/Thrift_homepage.png' alt='Thrift' width="1000" />
   
      但是不解决问题


1. 以动态扩展的形式优化 PHP 的性能瓶颈，用 PHP 加载 C++ 实现的方式绕开性能瓶颈
    - ebt 目前的解决方案，但是对 Facebook 这样历史包袱过重的源码仓库来说，性能瓶颈并不是 1-2 处小地方，而是不断累积的后果，并且 PHP 的扩展并不像 pybind 一样有比较成熟的加载方式

2. 优化 PHP 的解释器本身


### 如何优化 PHP 的解释器

1. 改进自己的源码
   - 用 PHP 写的 PHP 性能分析工具 [XHProf](https://github.com/phacility/xhprof)
   - 定位性能瓶颈处，优化代码逻辑，就像 leetcode 去做到 >99%
   - 优化的不够

2. 优化 PHP 的解释器实现
   - [Zend Engine](https://github.com/php/php-src/tree/master/Zend)
   - 将 PHP 编译为 `opcode` 然后执行 `opcode`
   - 优化 Zend 的性能代价太大，并且还要做到版本的向下兼容

3. 将 PHP 转为 C/C++，然后编译生成
   - Hiphop Compiler for PHP (Zhao, 2012)

   <img src='./images/hhvm_autogen.png' alt='hhvm_autogen' width="1000" />

    被认为是一种 Ahead of Time 的方式，能够完成非常多的代码优化（就像 LLVM 一样），但是一个问题就是无法正确支持 PHP 中的部分特性，如 `eval()`, `create_function()` 等

    > Support for eval is theoretically possible in HipHop by invoking an interpreter. However, we opted not to implement that because the use of eval exposes serious security problems and is therefore discouraged. (Zhao, 2012)

    就像是在 C++ 里面 `Py_Initialize()` 启动了 Python 环境，对内存消耗和代码优化都不是很友好

4. 实现一个虚拟机，将 PHP 源码转为当前平台的机器码执行 （JVM）
   - 如今的 HHVM


## HipHop Compiler for PHP （HPHPc）

### C++ 与 PHP 的特性区别

   <img src='./images/php_cpp_table.png' alt='PHP_cpp_table' width='600' />


### HPHPc 的编译设计

   <img src='./images/hphpc_phases.png' alt='hphpc_phases' width='600' />

```php
<?php

define("confName", "OOPSLA");
define("firstYear", 1986);

function year($edition) {
   return firstYear - 1 + $edition;
}

echo "Hello " . confName . "'" . year(27);
```

#### 1. 生成 AST

读取 PHP 的源码并生成对应的 AST

   <img src='./images/ast.png' alt='ast' width="1000" />


#### 2. 预分析

遍历 AST 并且记录所有符号的信息（类，函数，变量，全局 const），判断哪些符号是存在同名覆盖的情况需要考虑代码上下文。并且建立符号之间的依赖图，为后续的并行优化提供前置准备

#### 3. 预优化

处理不需要类型就能完成的油画，比如将函数转为内联函数，优化不必要的逻辑判断，移除不可能到达的代码片段等

#### 4. 判断类型

核心部分，基于 Damas-Milner constraint-based 算法判断不同符号的类型

<img src='./images/hphpc_types.png' alt='hphpc_types' width="600" />

`variant` 是 `any` 类型，所有符号在类型推断出类型前都是 `variant` 类型，在推断过程中如果成功判断为任何类型则覆盖 `variant`

#### 5. 后优化

在拥有类型之后，HipHop 编译器会根据类型信息优化包括简单的数值计算和逻辑判断在内的部分代码片段，然后再重新执行一次预优化阶段的优化逻辑

#### 6. 生成

最后编译器会遍历带有类型的，被优化后的 AST，并且生成对应的 C++ 代码，并且运行 gcc 编译 C++ 源码，生成的 C++ 部分包括

1. 类的头文件：每个 PHP 的类实现会生成对应的 C++ 头文件与对应的类声明
2. PHP 的头文件：每个 PHP 的文件（函数）会生成对应的 C++ 头文件与对应的声明
3. 具体实现文件：包含了一个或者多个声明类/函数的具体实现
4. 系统文件：不包含 PHP 的源码内容，但是会记录 Symbol Table

### 如何从 AST 生成 C++ 代码

#### 鸭子类型

除了正常标注类型的符号以外，仍然存在无法判断的 `variant` 类型，需要实现一个 `variant` 类型支持运行时处理的逻辑

#### 动态符号

指符号的具体指向需要运行时判断，如

```python
if SOME_CONDITION:
   x = 10
else:
   x = 'a'
```
对于这种情况，HipHop 用一张 *global symbol table* 记录了：

   - 全局变量
   - 动态申明的 constant
   - 函数体/类实现中的静态变量
   - 重复定义的函数体/类

所谓重复定义是指在不同文件中定义了同样的函数名/类名（在动态语言中这样做是合法的），GST 会在符号名后添加唯一后缀，然后根据具体 `#include` 语句引用的文件名导向不同后缀的同名函数/类实现。对于静态编译过程中出现的无法处理的**动态符号**，HipHop 会通过类似的逻辑生成临时的 *local symbol table*。

在具体处理实际的 variable 时，编译器会通过当前所处的上下文获取 LST 和 GST，从 table 中获取实际的指向。

同时，HipHop 还支持了动态添加实例属性，如

In [6]:
class SomeClass:

    def __init__(self):
        pass


some_instance = SomeClass()
some_instance.a = 10
print(some_instance.a)

10


为了实现这样的 feature，HipHop 还实现了一个 *property symbol table*，用于记录符号中的属性，当源码尝试访问实例/类属性时，会通过 PST 找到对应的符号。

但是 HPHPc 的痛点就是

- 无法支持 PHP 的动态方法 `eval()`, `create_function()` 等
- 并且编译后部署上线的源码重量会比 PHP 要大很多，对于2千万行的源码，这种硬性的成本是毁灭性的

## HHVM （HipHop Virtual Machine）

会什么虚拟机能够解决传统解释器的问题呢？

### JIT （Just In Time）

JVM，Numba 和 HHVM 都是对解释性语言的 jit 优化，但是尝试过 Numba 的我们也知道，jit 有可能并没有想象中的那么美好，对于一些简单的方法，jit 甚至还会拖累性能

In [8]:
def make_it_double(a: int) -> int:
    return a * 2

In [9]:
%%timeit
make_it_double(10)

38.7 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
import numba

make_it_double_numba = numba.jit(make_it_double)

/home/johnnyxcy/.local/lib/python3.8/site-packages/numba/core/errors.py:175: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [11]:
%%timeit
make_it_double_numba(10)

91.3 ns ± 0.569 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


从上面的表现可以发现 jit 并不是那么的美好，因为 jit 这个过程也是存在性能损耗的，所以这种简单的方法反而会比普通的解释器慢

<img src='./images/hphpc_vs_hhvm.png' alt='hphpc_vs_hhvm' width="600" />

安卓的目前的运行时方案 ART （Andriod Run Time）作为 Android 上的应用和部分系统服务使用的托管式运行时，采用的就是 HPHPc 的 AOT 方案，其前身 Dalvik 使用的 JIT 方案，但是 ART 虚拟机比 Dalvik 快了一倍多。

为了保证虚拟机 jit 的性能优化，Facebook 招聘了各路神仙

- Andrei Alexandrescu，『Modern C++ Design』和『C++ Coding Standards』的作者，C++ 领域无可争议的大神
- Keith Adams，负责过 VMware 核心架构，当年 VMware 就派他一人去和 Intel 进行技术合作，足以证明在 VMM 领域他有多了解了
- Drew Paroski，在微软参与过 .NET 虚拟机开发，改进了其中的 JIT
- Jason Evans，开发了 jemalloc，减少了 Firefox 一半的内存消耗
- Sara Golemon，『Extending and Embedding PHP』的作者，PHP 内核专家

### Interpreter

HHVM 在 parse 完 PHP 源码会生成一种 Bytecode （opcode），储存在 .hhvm.hhbc 文件中索引，在执行 Bytecode 时和 Zend 类似，将不同的字节码放到不同的函数中去实现 （Subroutine threading）。具体的生成逻辑在 `hphp/runtime/vm/hhbc.cpp`

因为重新实现了解释器，HHVM 比起 HPHPc 能够提供更加优异的兼容性，理论上可以做到对 PHP 所有特性的完美兼容，但是这样的性能还是走了 Zend 的老路子，并且对于动态类型，需要实现类似于如下的判断

```cpp
VMType::Variant VMExecutionContext::add(const VMType::Variant &left, const VMType::Variant &right) {
    if (this->GST[left.uid] == VMType::Int64 && this->GST[right.uid] == VMType::Int64) {
        return this->IntAddImpl->exec(left, right);
    }
    // TODO: some other impl
}
```

而我们知道这样的 if else 条件判断对 CPU 的执行优化是严重的拖累，另一个问题是需要从数据都是储存在对象中，作为 boxed structure 每次的间接获取地址也是有成本的，所以需要 jit 的实现来完成这些工作

### JIT Impl

其实本质上 jit 和 `eval()` 是类似的，只不过 jit eval 的不是源码字符串，而是不同平台下的机器码。HHVM 实现了一个 x64 的机器码生成器（HHBBC）。

常见的 jit 触发条件有两种

- trace：记录循环执行次数，如果超过一定数量就对这段代码进行 jit
- method：记录函数执行次数，如果超过一定数量就对这个函数内的代码片段进行 jit，如果过于 hot 就直接改为 inline

关于这两种方法哪种更好在 [Lambada](http://lambda-the-ultimate.org/node/3851) 上有个帖子引来了各路大神的讨论，尤其是 Mike Pall（LuaJIT 作者） 、Andreas Gal（Mozilla VP） 和 Brendan Eich（Mozilla CTO）都发表了很多自己的观点，两者的区别不仅仅是编译的范围，还有包括局部变量处理在内的很多细节方面都不太一样。

然而 HHVM 自创了一种叫做 tracelet 的方式，根据类型划分

<img src='./images/hhvm_tracelet.png' alt='hhvm_tracelet' width="1000" />

tracelet 将一个函数划分了 3 个部分，A，C用于处理入参 `$k` 是 integer 或者 string 两种类型的情况，B 用于处理返回值，因此 HHVM 的 jit 触发更像是与类型相关，具体如何分析和拆解 tracelet 的细节在 `hphp/runtime/vm/jit` 中，太深了就不需要了解了。

当然 HHVM 的 jit 优化还体现在很多地方，比如 如果 C 比 A 更加 hot（命中次数更多），HHVM 就会在 jit 的过程中倾向于把 C 放在 A 的前面，据说这样的操作提升了 14% 的性能，因为这样更容易提前命中需要相应的类型。

### Hack

hhvm/jit 的关键是类型，而猜测类型就像上面的例子一样是一个对 CPU 非常不友好的操作，所以 HHVM 的开发者就开始考虑为 PHP 添加类型支持，推出了一个新的语言 - Hack，通过静态类型指定让 HHVM 能够更好的提供性能优化。同时也能够让代码可读性大大提升，方便 IDE 提供更多帮助，在编码阶段减少很多没有必要的 bug。

## Conclusion

HPHPc / HHVM 作为两种为动态的脚本语言提供性能优化的方案都是有可取之处的，HPHPc 能够在静态编译的过程中完成各种可能的性能优化，比起 HHVM 的优化， HPHPc 的优化能够更加稳定，AOT 是一种非常成熟的优化方案

HHVM 则能够更加完美的兼容动态语言的特性，jit 是一种近年来发展迅速的技术，迭代更新快，能够一直保持对性能的优化。